In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
tb = soup.find('table', class_='wikitable')

In [4]:
# extract column headings
headings = []
headers = tb.find_all('th')
for i in range(len(headers)):
    headings.append(headers[i].get_text().replace('\n',''))

In [5]:
# initiate data structure
main = []
sub = []
values = tb.find_all('td')

# look through soup to extract data
for i in range(len(values)):
    val = values[i].get_text().replace('\n','')
    if i == 0:
        sub.append(val)
    if i%3 != 0:
        sub.append(val)
    if i%3 == 0 and i != 0:
        main.append(sub)
        sub = []
        sub.append(val)
    if i == range(len(values))[-1]:
        main.append(sub)

In [6]:
# create dataframe table with extracted data

import pandas as pd
df = pd.DataFrame(columns=headings, data=main)

In [7]:
# remove unassigned boroughs
df_clean = df[df['Borough'] != 'Not assigned']

In [8]:
# organise neighbourhoods 
df_clean = df_clean.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [9]:
# replace unassigned neighbourhoods
for i in range(len(df_clean)):
    if df_clean['Neighbourhood'][i] == 'Not assigned':
        df_clean['Neighbourhood'][i] = df_clean['Borough'][i]

In [10]:
df_clean.shape

(103, 3)

In [11]:
df_clean.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
